# Example / concept notebook for image viewer PSF photometry use case

The use case itself is detailed [here](image-viewer-psf-photometry.md).  This notebook gives an idea of how this use case would work.  Some/all of the code may not work, but the goal is for the notebook (or a very similar notebook) to work once the development work has been completed.


* Note: throughout this example, the assumption is that Roman data-ccess-related functionality all lives in `astroquery.mast` this does not *need* to be the case - some of the functionality could live in some other helper package.  But for the sake of example we will assume all-in-astroquery.

This example starts from the assumption that the user already has a known field.  While the science case outlined above is based on doing PSF photometry for a *whole cluster*, in this notebook, we will instead show the workflow for a *single target*.  In practice it's very similar, just with a lot more repetition in the cluster case that isn't necessary to understand the interactive elements.

In [ ]:
from astroquery.mast import Observations

from astropy import units as υ
from astropy.coordinates import SkyCoord
from astropy.table import Table

from photutils import psf

import roman_datamodels
import webbpsf  # which currently contains the Roman PSF models, although this may change
import jdaviz

In [ ]:
target_star_coordinates = SkyCoord(XXX*u.deg, YYY*u.deg, frame='icrs')  # coordinates known from some external source

In [ ]:
obses = Observations.query_region(target_star_coordinates, radius=5*υ.arcsec, obs_collection='roman')
products = Observations.get_product_list(obses)

... filter on a single exposure that is on the target star and create a `msk` that selects just that one product...

prod = products[msk][:1]
dled = Observations.download_products(prod)

Note that the above assumes that the data model for Roman allows for one to find a roman exposure by doing a `query_region` and even if the observation center is not that close to the target, it will still work.  If that is *not* intended, some other search will be needed that is more like `query_all_regions_that_cover_this_point` or the like.

Also note that it is assumed that `download_products` doesn't actually download anything if run in the RSP, just gives back the relevant path inside AWS.

In [ ]:
image = roman_datamodels.open(dled['Local Path'])

# the assumption here is that `roman_psf_model` magically knows how to interpret the metadata in `image`. 
#  `webbpsf` does *not* do this as far as I know, so it might instead need to be an example notebook that shows how to do this more verbosely
psf_model = webbpsf.roman_psf_model(image)


phot = psf.PSFPhotometry(psf_model)

target_table = Table()
target_table['x_init'] = image.meta.wcs.world_to_pixel(target_star_coordinates)[0]
target_table['y_init'] = image.meta.wcs.world_to_pixel(target_star_coordinates)[0]

psf_results = phot(image.data, target_table, init_params=target_table)

residual_image = phot.get_residual_image()

Now the user wants to examine the PSF fitting results, by blinking back and forth between the original and the subtracted image:

In [ ]:
imviz = jdaviz.Imviz()

imviz.load_data(image, data_label='original_image')
imviz.load_data(residual_image, data_label='residual')

imviz.show()

Now by clicking on the "blink" option in imviz and hitting the "b" key, the image will blink between the two, so the user can examine how good the fit is.  If need be, the user can then do something like:

In [ ]:
some_second_target_table = psf_results.copy()
some_second_target_table ... add more sources to measure that were revealed in the residual image ...

phot(residual_image, some_second_target_table, init_params=target_table)

residual_image2 = phot.get_residual_image()

imviz.load_data(residual_image2, data_label='residual')

And now the user can blink between all three.  Rinse and repeat until satisfied!